<center><img src="img/header.png"></center>

<h2><center>Лекция 10: Нейронные сети. Основы. (Тыкаем в керас 3)</center></h2>
<h3><center>Шестаков Андрей</center></h3>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-talk')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

import seaborn as sn
    
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras import regularizers as reg

In [ ]:
# Uncomment if you are using colab
# !mkdir ./data

# !wget https://raw.githubusercontent.com/shestakoff/sphere-ml-intro/master/2020/lecture10-dl-intro/data/bikesharing.csv -O ./data/bikesharing.csv

Рассмотрим датасет для оценки спроса на прокатные велосипеды:

In [ ]:
df = pd.read_csv('./data/bikesharing.csv')
df.head()


* dteday - date
* yr - year code (2011 - 0, 2012 - 1)
* month - month code
* hr - hour code
* holiday - whether the day is considered a holiday
* weekday - weekday
* workingday - whether the day is neither a weekend nor holiday
* weather -
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* temp - temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* cnt - number of total rentals (Dependent Variable)


In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(18, 15)
sn.boxplot(data=df, y="cnt", orient="v", ax=axes[0][0])
sn.boxplot(data=df, y="cnt", x="mnth", orient="v", ax=axes[0][1])
sn.boxplot(data=df, y="cnt", x="hr", orient="v", ax=axes[1][0])
sn.boxplot(data=df, y="cnt", x="weekday", orient="v", ax=axes[1][1])

axes[0][0].set(ylabel='Count', title="Box Plot On Count")
axes[0][1].set(xlabel='Month', ylabel='Count', title="Box Plot On Count Across Year")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count', title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count', title="Box Plot On Count Across Weekdays")

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sn.regplot(x="temp", y="cnt", data=df, ax=ax1)
sn.regplot(x="windspeed", y="cnt", data=df, ax=ax2)
sn.regplot(x="hum", y="cnt", data=df, ax=ax3)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

Попробуем сделать пайплайн

* Все числовые признаки мы прогоним через StandartScaler
* Бинарные признаки оставим как есть
* Категориальные признаки - OHE

In [ ]:
X = df.loc[:, ['yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday',
               'weathersit', 'temp', 'hum', 'windspeed']].values
y = np.log(df.loc[:, 'cnt'].values+1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
preproc = ColumnTransformer(
    [('one-hot', OneHotEncoder(), [1,2,4,6]),
     ('continious', StandardScaler(), [7,8,9])], 
    remainder='passthrough', 
    sparse_threshold=0 ) # Всегда возвращать dense

In [ ]:
X_train_preproc = preproc.fit_transform(X_train)
X_valid_preproc = preproc.transform(X_valid)

In [ ]:
# Далее описываем модель по слоям
# В Sequential каждый слой идет на вход в следующий
model = keras.Sequential([
    L.Dense(50, input_shape=[X_train_preproc.shape[1]],
            activation='???', 
            kernel_regularizer=reg.l2(0.01), name='hidden'),
    L.Dense(1, activation='???', kernel_regularizer=reg.l2(0.01),
            name='output')
])

In [ ]:
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam() # Задаем оптимизатор

# Компилируем модель (keras строит граф вычислений, модель иницилизируется)
model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae', 'mse']) 



In [ ]:
y_hat = model.predict(X_valid_preproc) # Она уже может предсказывать, правда пока белеберду

In [ ]:
# Запускаем обучение, лог обучения будет записываться в переменную metrics
metrics = \
model.fit(x=X_train_preproc, 
          y=y_train, 
          batch_size=512, 
          epochs=200, 
          validation_data=[X_valid_preproc, y_valid])

In [ ]:
plt.plot(metrics.history['loss'][2:], label='train_mae')
plt.plot(metrics.history['val_loss'][2:], label='val_mae')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

In [ ]:
y_hat = model.predict(X_valid_preproc)

In [ ]:
plt.scatter(np.expm1(y_hat), np.expm1(y_valid))
plt.plot(np.expm1(y_valid), np.expm1(y_valid), c='red')

## Добавим Dropout!

In [ ]:
model = keras.Sequential([
    L.Dense(50, input_shape=[X_train_preproc.shape[1]],
            activation='???',name='hidden'),
    L.Dropout(0.2), 
    L.Dense(1, activation='???',
            name='output')
])

In [ ]:
optimizer = keras.optimizers.Adam()

model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae', 'mse'])


In [ ]:
y_hat = model.predict(X_valid_preproc)

In [ ]:
metrics = \
model.fit(x=X_train_preproc, 
          y=y_train, batch_size=512, 
          epochs=200, validation_data=[X_valid_preproc, y_valid])

In [ ]:
plt.plot(metrics.history['loss'][5:], label='train_mae')
plt.plot(metrics.history['val_loss'][5:], label='val_mae')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

### Добавим эмбеддингов!

Нейронные сети предпочитают нормированные и плотные данные,а мы имеем разреженные вектора.

Введем эмбединги для категориальных переменных, которые тоже будут обучаться вместе с остальными весами сети.

Для этого придется немного похимичить и поработать с [Keras Functional API](https://keras.io/getting-started/functional-api-guide/).

Попробуем получить что-то такое:
<center><img src='./img/categ_emb.png'></center>

In [ ]:
columns = \
np.array(['yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'hum', 'windspeed'])

In [ ]:
categ_variables = [1,2,4] # Колонки, для которых будут строится эмбединги
asis_variables = [0,3,7,8,9] # Колонки, которые пойдут почти без изменений

print(columns[categ_variables])
print(columns[asis_variables])

Сейчас мы разделим исходую матрицу `X` на кусочки:
* по кусочку на каждую категориальную переменную
* кусочек на все остельные переменные (бинарные + нормализованные числовые)

In [ ]:
# Обучим нормализатор
scaler = StandardScaler()
scaler.fit(X_train[:, [7,8,9]])

X_train_asis = np.c_[X_train[:,[0,3]], scaler.transform(X_train[:, [7,8,9]])]
X_train_month = X_train[:,[categ_variables[0]]]-1 # -1 чтобы значения были от 0 до n-1
X_train_hour = X_train[:,[categ_variables[1]]]
X_train_weekday = X_train[:,[categ_variables[2]]]

X_valid_asis = np.c_[X_valid[:,[0,3]], scaler.transform(X_valid[:, [7,8,9]])]
X_valid_month = X_valid[:, [categ_variables[0]]]-1  # -1 чтобы значения были от 0 до n-1
X_valid_hour = X_valid[:, [categ_variables[1]]]
X_valid_weekday = X_valid[:,[categ_variables[2]]]

In [ ]:
# И теперь буквально тоже по кирпичикам описываем наши компоненты

# Сначала входы нейронной сети:
cont_input = L.Input(shape=(5,), name='cont_variables')
month_input = L.Input(shape=(1,), dtype='int32')
weekday_input = L.Input(shape=(1,), dtype='int32')
hour_input = L.Input(shape=(1,), dtype='int32')

# Эмбеддинги
month_embedding = L.Embedding(input_dim=12, # Количество уникальных значений
                              output_dim=4, # Длина эмбединга
                              name='month_emb')(month_input) # На вход подаем input
month_embedding = L.Flatten()(month_embedding) # Размазываем слой

weekday_embedding = ...

hour_embedding = ...

# Конкатенируем слой на эмбедингах
all_embeddings = L.concatenate(...)

# Полносвязные слой на эмбедингах:
dense1 = L.Dense(10, activation='tanh')(...)

# Конкатенируем c остальными признаками
all_features = L.concatenate(...)

# Еще полносвязные слой
dense2 = L.Dense(6, activation='tanh')(...)

# Выходной слой
output = L.Dense(1, activation='linear')(...)

# Формируем модель
model = keras.Model(inputs=[cont_input, month_input, hour_input, weekday_input], 
                    outputs=output)


In [ ]:
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam()

model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae', 'mse'])


In [ ]:
metrics = \
model.fit(x=[X_train_asis, X_train_month, X_train_hour, X_train_weekday],
          y=y_train, batch_size=512, 
          epochs=500, 
          validation_data=[[X_valid_asis, X_valid_month, X_valid_hour, X_valid_weekday], y_valid])

In [ ]:
plt.plot(metrics.history['loss'][5:], label='train_mae')
plt.plot(metrics.history['val_loss'][5:], label='val_mae')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

In [ ]:
y_hat = model.predict([X_valid_asis, X_valid_month, X_valid_hour, X_valid_weekday])

In [ ]:
plt.scatter(np.expm1(y_hat), np.expm1(y_valid))
plt.plot(np.expm1(y_valid), np.expm1(y_valid))

### Посмотрим на эмбеддинги

In [ ]:
model.layers

In [ ]:
emb = model.layers[4]
emb.name

In [ ]:
weights = emb.get_weights()[0]
weights

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
H = linkage(weights, method='average', metric='cosine')
d = dendrogram(H)